# Building And Testing A Complete Trading System

In [7]:
import yfinance as yf
import pandas_ta as pa
import plotly.graph_objects as go
import numpy as np
import pandas as pd

def get_data(symbol: str):
    # data = yf.download(tickers=symbol, period='1000d', interval='1d')
    data = pd.read_csv("BTCUSD_Candlestick_1_D_ASK_08.05.2017-16.10.2021.csv")

    data.reset_index(inplace=True)
    return data
# Get the data
data = get_data('BTC-USD')

In [8]:
data

,index,Gmt time,Open,High,Low,Close,Volume
0,0,07.05.2017 21:00:00.000,1559,1681,1557,1658,1.872044e+06
1,1,08.05.2017 21:00:00.000,1658,1753,1626,1747,1.330136e+06
2,2,09.05.2017 21:00:00.000,1713,1769,1677,1752,1.916895e+06
3,3,10.05.2017 21:00:00.000,1784,1887,1777,1857,1.613521e+06
4,4,11.05.2017 21:00:00.000,1836,1836,1651,1692,2.550428e+06
...,...,...,...,...,...,...,...
1301,1301,12.10.2021 21:00:00.000,55433,57543,54328,57047,2.013182e+03
1302,1302,13.10.2021 21:00:00.000,57078,58568,56906,57505,2.001020e+03
1303,1303,14.10.2021 21:00:00.000,57533,62922,56940,62587,2.235774e+03
1304,1304,15.10.2021 21:00:00.000,62587,62608,60223,61077,1.943311e+03


## 1- Add rejection signal

In [9]:
def identify_rejection(data):
    # Create a new column for rejection signal
    data['rejection'] = data.apply(lambda row: 2 if (
        ( (min(row['Open'], row['Close']) - row['Low']) > (1.5 * abs(row['Close'] - row['Open']))) and 
        (row['High'] - max(row['Close'], row['Open'])) < (0.8 * abs(row['Close'] - row['Open'])) and 
        (abs(row['Open'] - row['Close']) > row['Open'] * 0.001)
    ) else 1 if (
        (row['High'] - max(row['Open'], row['Close'])) > (1.5 * abs(row['Open'] - row['Close'])) and 
        (min(row['Close'], row['Open']) - row['Low']) < (0.8 * abs(row['Open'] - row['Close'])) and 
        (abs(row['Open'] - row['Close']) > row['Open'] * 0.001)
    ) else 0, axis=1)

    return data

data = identify_rejection(data)


In [10]:
data[data["rejection"]!=0]

,index,Gmt time,Open,High,Low,Close,Volume,rejection
26,26,12.06.2017 21:00:00.000,2657,2788,2638,2702,2.226144e+06,1
28,28,14.06.2017 21:00:00.000,2553,2559,2077,2363,2.589326e+06,2
38,38,28.06.2017 21:00:00.000,2595,2605,2503,2559,2.302025e+06,2
41,41,03.07.2017 21:00:00.000,2558,2642,2545,2577,1.820193e+06,1
80,80,27.08.2017 21:00:00.000,4366,4370,4191,4340,2.018315e+06,2
...,...,...,...,...,...,...,...,...
1243,1243,15.08.2021 21:00:00.000,46767,48088,45720,46141,1.292448e+03,1
1266,1266,07.09.2021 21:00:00.000,46931,47405,44494,46116,2.135367e+03,2
1283,1283,24.09.2021 21:00:00.000,43028,43264,41754,42720,1.058906e+03,2
1284,1284,25.09.2021 21:00:00.000,42720,43776,40860,43686,1.162478e+03,2


In [11]:
def pointpos(x, xsignal):
    if x[xsignal]==1:
        return x['High']+1e-4
    elif x[xsignal]==2:
        return x['Low']-1e-4
    else:
        return np.nan

def plot_with_signal(dfpl):

    fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                    open=dfpl['Open'],
                    high=dfpl['High'],
                    low=dfpl['Low'],
                    close=dfpl['Close'])])

    fig.update_layout(
        autosize=False,
        width=1000,
        height=800, 
        paper_bgcolor='black',
        plot_bgcolor='black')
    fig.update_xaxes(gridcolor='black')
    fig.update_yaxes(gridcolor='black')
    fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                    marker=dict(size=8, color="MediumPurple"),
                    name="Signal")
    fig.show()

data['pointpos'] = data.apply(lambda row: pointpos(row,"rejection"), axis=1)
plot_with_signal(data[10:110])

## 2- Support and Resistance FUNCTIONS

In [12]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    if ( df1.Low[l-n1:l].min() < df1.Low[l] or
        df1.Low[l+1:l+n2+1].min() < df1.Low[l] ):
        return 0
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    if ( df1.High[l-n1:l].max() > df1.High[l] or
       df1.High[l+1:l+n2+1].max() > df1.High[l] ):
        return 0
    return 1

## 3- Close to resistance and support testing

In [13]:
def closeResistance(l, levels, lim, df):
    if len(levels) == 0:
        return 0
    c1 = abs(df['High'][l] - min(levels, key=lambda x: abs(x - df['High'][l]))) <= lim
    c2 = abs(max(df['Open'][l], df['Close'][l]) - min(levels, key=lambda x: abs(x - df['High'][l]))) <= lim
    c3 = min(df['Open'][l], df['Close'][l]) < min(levels, key=lambda x: abs(x - df['High'][l]))
    c4 = df['Low'][l] < min(levels, key=lambda x: abs(x - df['High'][l]))
    if (c1 or c2) and c3 and c4:
        return min(levels, key=lambda x: abs(x - df['High'][l]))
    else:
        return 0

def closeSupport(l, levels, lim, df):
    if len(levels) == 0:
        return 0
    c1 = abs(df['Low'][l] - min(levels, key=lambda x: abs(x - df['Low'][l]))) <= lim
    c2 = abs(min(df['Open'][l], df['Close'][l]) - min(levels, key=lambda x: abs(x - df['Low'][l]))) <= lim
    c3 = max(df['Open'][l], df['Close'][l]) > min(levels, key=lambda x: abs(x - df['Low'][l]))
    c4 = df['High'][l] > min(levels, key=lambda x: abs(x - df['Low'][l]))
    if (c1 or c2) and c3 and c4:
        return min(levels, key=lambda x: abs(x - df['Low'][l]))
    else:
        return 0

def is_below_resistance(l, level_backCandles, level, df):
    return df.loc[l-level_backCandles:l-1, 'High'].max() < level

def is_above_support(l, level_backCandles, level, df):
    return df.loc[l-level_backCandles:l-1, 'Low'].min() > level

In [14]:
def check_candle_signal(l, n1, n2, levelbackCandles, windowbackCandles, df):
    ss = []
    rr = []
    for subrow in range(l-levelbackCandles, l-n2+1):
        if support(df, subrow, n1, n2):
            ss.append(df.Low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.High[subrow])

    ss.sort() #keep lowest support when popping a level
    for i in range(1,len(ss)):
        if(i>=len(ss)):
            break
        if abs(ss[i]-ss[i-1])/ss[i]<=0.001: # merging close distance levels
            ss.pop(i)

    rr.sort(reverse=True) # keep highest resistance when popping one
    for i in range(1,len(rr)):
        if(i>=len(rr)):
            break
        if abs(rr[i]-rr[i-1])/rr[i]<=0.001: # merging close distance levels
            rr.pop(i)

    #----------------------------------------------------------------------
    # joined levels
    rrss = rr+ss
    rrss.sort()
    for i in range(1,len(rrss)):
        if(i>=len(rrss)):
            break
        if abs(rrss[i]-rrss[i-1])/rrss[i]<=0.001: # merging close distance levels
            rrss.pop(i)
    cR = closeResistance(l, rrss, df.Close[l]*0.003, df)
    cS = closeSupport(l, rrss, df.Close[l]*0.003, df)
    #----------------------------------------------------------------------

    # cR = closeResistance(l, rr, 150e-5, df)
    # cS = closeSupport(l, ss, 150e-5, df)
    #print(rrss,df.Close*0.002)
    if (df.rejection[l] == 1 and cR and is_below_resistance(l,windowbackCandles,cR, df)):
        return 1
    elif(df.rejection[l] == 2 and cS and is_above_support(l,windowbackCandles,cS, df)):
        return 2
    else:
        return 0

In [15]:
from tqdm import tqdm

n1 = 8
n2 = 8
levelbackCandles = 60
windowbackCandles = n2

signal = [0 for i in range(len(data))]

for row in tqdm(range(levelbackCandles+n1, len(data)-n2)):
    signal[row] = check_candle_signal(row, n1, n2, levelbackCandles, windowbackCandles, data)

data["signal"] = signal

  0%|          | 0/1230 [00:00<?, ?it/s]

100%|██████████| 1230/1230 [00:16<00:00, 74.90it/s] 


In [16]:
data[data["signal"]!=0]

,index,Gmt time,Open,High,Low,Close,Volume,rejection,pointpos,signal
1128,1128,22.04.2021 21:00:00.000,51625,52252,47573,50621,2270.844,2,47572.9999,2


In [17]:
data['pointpos'] = data.apply(lambda row: pointpos(row,"signal"), axis=1)
plot_with_signal(data[750:950])

## 4- Backtesting

In [18]:
data.set_index("Date", inplace=True)

KeyError: "None of ['Date'] are in the columns"

In [ ]:
data

,Open,High,Low,Close,Adj Close,Volume,rejection,pointpos,signal
Date,,,,,,,,,
2021-01-27,32564.029297,32564.029297,29367.138672,30432.546875,30432.546875,62576762015,0,NaN,0
2021-01-28,30441.041016,33858.312500,30023.207031,33466.097656,33466.097656,76517157706,0,NaN,0
2021-01-29,34318.671875,38406.261719,32064.814453,34316.386719,34316.386719,117894572511,0,NaN,0
2021-01-30,34295.933594,34834.707031,32940.187500,34269.523438,34269.523438,65141828798,0,NaN,0
2021-01-31,34270.878906,34288.332031,32270.175781,33114.359375,33114.359375,52754542671,0,NaN,0
...,...,...,...,...,...,...,...,...,...
2023-10-19,28332.416016,28892.474609,28177.988281,28719.806641,28719.806641,14448058195,0,NaN,0
2023-10-20,28732.812500,30104.085938,28601.669922,29682.949219,29682.949219,21536125230,0,NaN,0
2023-10-21,29683.380859,30287.482422,29481.751953,29918.412109,29918.412109,11541146996,0,NaN,0


In [ ]:
data['ATR'] = pa.atr(high=data.High, low=data.Low, close=data.Close, length=14)
data['RSI'] = pa.rsi(data.Close, length=5)

In [ ]:
def SIGNAL():
    return data.signal

In [ ]:
#A new strategy needs to extend Strategy class and override its two abstract methods: init() and next().
#Method init() is invoked before the strategy is run. Within it, one ideally precomputes in efficient, 
#vectorized manner whatever indicators and signals the strategy depends on.
#Method next() is then iteratively called by the Backtest instance, once for each data point (data frame row), 
#simulating the incremental availability of each new full candlestick bar.

#Note, backtesting.py cannot make decisions / trades within candlesticks — any new orders are executed on the
#next candle's open (or the current candle's close if trade_on_close=True). 
#If you find yourself wishing to trade within candlesticks (e.g. daytrading), you instead need to begin 
#with more fine-grained (e.g. hourly) data.

### 4.1- Using Fixed SL and TP rules

In [ ]:
# Trader fixed SL and TP
from backtesting import Strategy, Backtest

class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        self.ratio = 2
        self.risk_perc = 0.1

    def next(self):
        super().next() 
        if self.signal1==2:
            sl1 = self.data.Close[-1] - self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] + (self.data.Close[-1]*self.risk_perc)*self.ratio
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] - (self.data.Close[-1]*self.risk_perc)*self.ratio
            self.sell(sl=sl1, tp=tp1)
bt = Backtest(data, MyCandlesStrat, cash=100_000, commission=.02)
stat = bt.run()
stat

Start                     2021-01-27 00:00:00
End                       2023-10-23 00:00:00
Duration                    999 days 00:00:00
Exposure Time [%]                        18.1
Equity Final [$]                142788.814766
Equity Peak [$]                 142788.814766
Return [%]                          42.788815
Buy & Hold Return [%]                4.032554
Return (Ann.) [%]                   13.884175
Volatility (Ann.) [%]               25.641494
Sharpe Ratio                         0.541473
Sortino Ratio                         0.98902
Calmar Ratio                          0.51796
Max. Drawdown [%]                  -26.805477
Avg. Drawdown [%]                   -6.684569
Max. Drawdown Duration      227 days 00:00:00
Avg. Drawdown Duration       43 days 00:00:00
# Trades                                    6
Win Rate [%]                        66.666667
Best Trade [%]                      18.378618
Worst Trade [%]                     -12.33479
Avg. Trade [%]                    

In [ ]:
bt.plot()

f:\Python\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

f:\Python\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p1331', ...)

### 4.2- Using the RSI for Exit Signals

In [ ]:
from backtesting import Strategy, Backtest

class MyCandlesStrat(Strategy):
    ratio = 1.5
    risk_perc = 0.1  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #self.ratio
        #self.risk_perc

    def next(self):
        super().next()
        
        if len(self.trades)>0:
            if self.trades[-1].is_long and self.data.RSI[-1]>=80:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=20:
                self.trades[-1].close()

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] + (self.data.Close[-1]*self.risk_perc)*self.ratio
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.Close[-1] + self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] - (self.data.Close[-1]*self.risk_perc)*self.ratio
            self.sell(sl=sl1, tp=tp1)
bt = Backtest(data, MyCandlesStrat, cash=100_000, commission=.05)
stat = bt.run()
stat

Start                     2021-01-27 00:00:00
End                       2023-10-23 00:00:00
Duration                    999 days 00:00:00
Exposure Time [%]                        14.1
Equity Final [$]                137872.424902
Equity Peak [$]                 137872.424902
Return [%]                          37.872425
Buy & Hold Return [%]                4.032554
Return (Ann.) [%]                   12.437002
Volatility (Ann.) [%]               21.456755
Sharpe Ratio                         0.579631
Sortino Ratio                        1.144943
Calmar Ratio                         1.070366
Max. Drawdown [%]                  -11.619395
Avg. Drawdown [%]                   -6.296726
Max. Drawdown Duration      230 days 00:00:00
Avg. Drawdown Duration       59 days 00:00:00
# Trades                                    7
Win Rate [%]                        85.714286
Best Trade [%]                      10.538669
Worst Trade [%]                     -1.949456
Avg. Trade [%]                    

In [ ]:
bt.plot()

f:\Python\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

f:\Python\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p1691', ...)

In [ ]:
# Define a range of values to test for each parameter
param_grid = {'ratio': list(np.arange(1.5, 3.5, 0.5)), 'risk_perc': list(np.arange(0.06, 0.2, 0.02))}
# Run the optimization
res = bt.optimize(**param_grid, random_state=5)

In [ ]:
# Print the best results and the parameters that lead to these results
print("Best result: ", res['Return [%]'])
print("Parameters for best result: ", res['_strategy'])

Best result:  37.87242490234372
Parameters for best result:  MyCandlesStrat(ratio=1.5,risk_perc=0.1)


### 4.3- ATR based SL and TP

In [ ]:
# ATR related SL and TP
from backtesting import Strategy, Backtest
import numpy as np

class MyCandlesStrat(Strategy): 
    atr_f = 3
    ratio_f = 2
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1==2:
            sl1 = self.data.Close[-1] - self.data.ATR[-1]*self.atr_f
            tp1 = self.data.Close[-1] + self.data.ATR[-1]*self.ratio_f*self.atr_f
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + self.data.ATR[-1]*self.atr_f
            tp1 = self.data.Close[-1] - self.data.ATR[-1]*self.ratio_f*self.atr_f
            self.sell(sl=sl1, tp=tp1)
bt = Backtest(data, MyCandlesStrat, cash=100_000, commission=.000)
stat = bt.run()
stat

Start                     2021-01-27 00:00:00
End                       2023-10-23 00:00:00
Duration                    999 days 00:00:00
Exposure Time [%]                        30.0
Equity Final [$]                131344.601953
Equity Peak [$]                  150496.75039
Return [%]                          31.344602
Buy & Hold Return [%]                4.032554
Return (Ann.) [%]                   10.463923
Volatility (Ann.) [%]               29.445125
Sharpe Ratio                          0.35537
Sortino Ratio                        0.595034
Calmar Ratio                         0.328157
Max. Drawdown [%]                  -31.886966
Avg. Drawdown [%]                   -4.634481
Max. Drawdown Duration      400 days 00:00:00
Avg. Drawdown Duration       37 days 00:00:00
# Trades                                    6
Win Rate [%]                             50.0
Best Trade [%]                      26.795577
Worst Trade [%]                    -12.568627
Avg. Trade [%]                    

In [ ]:
bt.plot()

f:\Python\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

f:\Python\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p2051', ...)

### 4.4- Trail Stop

In [ ]:
#fixed distance Trailing SL
from backtesting import Strategy, Backtest

class MyCandlesStrat(Strategy):
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        sltr=self.data.Close[-1]*0.02

        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - sltr)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + sltr) 
        
        if self.signal1==2 and len(self.trades)==0: 
            sl1 = self.data.Close[-1] - sltr
            self.buy(sl=sl1)
        elif self.signal1==1 and len(self.trades)==0: 
            sl1 = self.data.Close[-1] + sltr
            self.sell(sl=sl1)


bt = Backtest(data, MyCandlesStrat, cash=100_000, commission=.000)
stat = bt.run()
stat

Start                     2021-01-27 00:00:00
End                       2023-10-23 00:00:00
Duration                    999 days 00:00:00
Exposure Time [%]                         4.2
Equity Final [$]                118635.360469
Equity Peak [$]                 123280.318594
Return [%]                           18.63536
Buy & Hold Return [%]                4.032554
Return (Ann.) [%]                    6.435907
Volatility (Ann.) [%]                7.235593
Sharpe Ratio                         0.889479
Sortino Ratio                        2.294279
Calmar Ratio                         1.287354
Max. Drawdown [%]                    -4.99933
Avg. Drawdown [%]                   -2.600717
Max. Drawdown Duration      332 days 00:00:00
Avg. Drawdown Duration      156 days 00:00:00
# Trades                                    7
Win Rate [%]                        42.857143
Best Trade [%]                      24.949697
Worst Trade [%]                     -2.081687
Avg. Trade [%]                    

In [ ]:
bt.plot()

f:\Python\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

f:\Python\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p2411', ...)

In [ ]:
#ATR based Trailing Stop
from backtesting import Strategy, Backtest

class MyCandlesStrat(Strategy):
    atr_f = 0.6
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        self.sltr=0

    def next(self):
        super().next()
        
        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - self.sltr)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + self.sltr)

        if self.signal1==2 and len(self.trades)==0: 
            self.sltr=self.data.ATR[-1]/self.atr_f
            sl1 = self.data.Close[-1] - self.data.ATR[-1]/self.atr_f
            self.buy(sl=sl1)
        elif self.signal1==1 and len(self.trades)==0: 
            self.sltr=self.data.ATR[-1]/self.atr_f
            sl1 = self.data.Close[-1] + self.data.ATR[-1]/self.atr_f
            self.sell(sl=sl1)
bt = Backtest(data, MyCandlesStrat, cash=100_000, commission=.000)
stat = bt.run()
stat

Start                     2021-01-24 00:00:00
End                       2023-10-20 00:00:00
Duration                    999 days 00:00:00
Exposure Time [%]                         8.5
Equity Final [$]                114399.742824
Equity Peak [$]                 124148.873143
Return [%]                          14.399743
Buy & Hold Return [%]               -8.648288
Return (Ann.) [%]                    5.032848
Volatility (Ann.) [%]               12.076801
Sharpe Ratio                         0.416737
Sortino Ratio                        0.726958
Calmar Ratio                         0.399561
Max. Drawdown [%]                  -12.595935
Avg. Drawdown [%]                   -5.345708
Max. Drawdown Duration      325 days 00:00:00
Avg. Drawdown Duration      130 days 00:00:00
# Trades                                    7
Win Rate [%]                        57.142857
Best Trade [%]                      24.433671
Worst Trade [%]                     -7.018164
Avg. Trade [%]                    

## 5- Lot sizing and trade management

In [ ]:
class MyCandlesStrat(Strategy):
    lotsize = 1 
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        self.ratio = 1.
        self.risk_perc = 0.1

    def next(self):
        super().next() 
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] + (self.data.Close[-1]*self.risk_perc)*self.ratio*0.8
            tp2 = self.data.Close[-1] + (self.data.Close[-1]*self.risk_perc)*self.ratio*1.2
            self.buy(sl=sl1, tp=tp1, size=self.lotsize)
            self.buy(sl=sl1, tp=tp2, size=self.lotsize)
        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.Close[-1] + self.data.Close[-1]*self.risk_perc
            tp1 = self.data.Close[-1] - (self.data.Close[-1]*self.risk_perc)*self.ratio*0.8
            tp2 = self.data.Close[-1] - (self.data.Close[-1]*self.risk_perc)*self.ratio*1.2
            self.sell(sl=sl1, tp=tp1, size=self.lotsize)
            self.sell(sl=sl1, tp=tp2, size=self.lotsize)
bt = Backtest(data, MyCandlesStrat, cash=100_000, margin=1/1, commission=.05)
stat = bt.run()
stat

Start                     2021-01-27 00:00:00
End                       2023-10-23 00:00:00
Duration                    999 days 00:00:00
Exposure Time [%]                        14.8
Equity Final [$]                115220.358555
Equity Peak [$]                 115220.358555
Return [%]                          15.220359
Buy & Hold Return [%]                4.032554
Return (Ann.) [%]                    5.307224
Volatility (Ann.) [%]               12.232895
Sharpe Ratio                         0.433849
Sortino Ratio                        0.759777
Calmar Ratio                         0.526458
Max. Drawdown [%]                  -10.080995
Avg. Drawdown [%]                   -3.932764
Max. Drawdown Duration      228 days 00:00:00
Avg. Drawdown Duration       33 days 00:00:00
# Trades                                   13
Win Rate [%]                        92.307692
Best Trade [%]                        7.38121
Worst Trade [%]                    -14.284073
Avg. Trade [%]                    